In [ ]:
import pandas as pd
import numpy  as np
import networkx as nx
from math import sin, cos, pi, sqrt
import random
import time
import rtsvg
rt = rtsvg.RACETrack()

_n_paths_         = 30
_n_circles_       = 20
_radius_min_      = 20
_radius_max_      = 30
_min_circle_sep_  = 30
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def createCircleDataset(n_circles=_n_circles_, n_paths=_n_paths_, radius_min=_radius_min_, radius_max=_radius_max_, min_circle_sep=_min_circle_sep_, radius_inc_test=_radius_inc_test_):
    circle_geoms = []
    def circleOverlaps(cx, cy, r):
        for _geom_ in circle_geoms:
            dx, dy = _geom_[0] - cx, _geom_[1] - cy
            d      = sqrt(dx*dx+dy*dy)
            if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
                return True
        return False
    def findOpening():
        _max_attempts_ = 100
        attempts  = 0
        cx, cy, r = random.randint(radius_max+min_circle_sep, 600-radius_max-min_circle_sep), \
                    random.randint(radius_max+min_circle_sep, 400-radius_max-min_circle_sep), random.randint(radius_min,radius_max)
        while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
            cx, cy, r = random.randint(radius_max+min_circle_sep, 600-radius_max-min_circle_sep), \
                        random.randint(radius_max+min_circle_sep, 400-radius_max-min_circle_sep), random.randint(radius_min,radius_max)
            attempts += 1
        if attempts == _max_attempts_:
            return None
        return cx, cy, r

    # Randomize the circles
    for i in range(n_circles):
        to_unpack = findOpening()
        if to_unpack is not None:
            cx, cy, r = to_unpack
            circle_geoms.append((cx,cy,r))

    # Randomize the entry point
    c0         = random.randint(0, len(circle_geoms)-1)
    cx, cy, r  = circle_geoms[c0]
    a0         = random.random() * 2 * pi
    entry_pt   = (cx+(r+_radius_inc_test_+0.5)*cos(a0),cy+(r+_radius_inc_test_+0.5)*sin(a0),c0)
                
    # Randomize the exit points
    exit_pts = []
    for i in range(n_paths):
        c1 = random.randint(0,len(circle_geoms)-1)
        while c1 == c0:
            c1 = random.randint(0,len(circle_geoms)-1)
        cx, cy, r  = circle_geoms[c1]
        a1         = random.random() * 2 * pi
        exit_pts.append((cx+(r+radius_inc_test+0.5)*cos(a1),cy+(r+radius_inc_test+0.5)*sin(a1),c1))

    return entry_pt, exit_pts, circle_geoms

_entry_pt_,_exit_pts_,_circle_geoms_ = createCircleDataset()

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#404040" fill="#a0a0a0" fill-opacity="0.2" />'
# Render Entry Points
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="3" stroke="#00af00" fill="#00af00" />'
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="6" stroke="#000000" fill="none" />'
# Render Exit Points
for i in range(len(_exit_pts_)):
    _exit_  = _exit_pts_[i]
    _color_ = rt.co_mgr.getColor(i) 
    svg += f'<circle cx="{_exit_[0]}" cy="{_exit_[1]}" r="3" stroke="{_color_}" fill="{_color_}" />'
svg_base = svg
rt.displaySVG(svg_base + '</svg>')

In [ ]:
lsvg = lambda seg: f' x1="{seg[0][0]}" y1="{seg[0][1]}" x2="{seg[1][0]}" y2="{seg[1][1]}" '
#
# circularPathRouter() - route exits to a single entry around circles
# - all points need to be at least circle_radius + radius_inc_test + 1.0 from the circle centers...
#
def circularPathRouter(entry_pt,                  # (x,y,circle_i) -- where circle_i is the circle index from circle_geoms
                       exit_pts,                  # [(x,y,circle_i),(x,y,circle_i),(x,y,circle_i), ...] -- where circle_i is the circle index from circle_geoms
                       circle_geoms,              # [(cx,cy,r),(cx,cy,r), ...]
                       escape_px          = 5,    # length to push the exit points (and entry point) away from circle
                       min_circle_sep     = 30,   # minimum distance between circles
                       half_sep           = 15,   # needs to be more than the radius_inc_test ... half separation (but doesn't have to be)
                       radius_inc_test    = 4,    # for routing around circles, how much to test with
                       radius_start       = 5,    # needs to be more than the radius_inc_test ... less than the min_circle_sep
                       max_pts_per_node   = 50,   # maximum points per node for the xy quad tree
                       merge_distance_min = 5):   # minimum distance necessary to merge a path into an already exiting path
    _svg_ = ['<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />']
    for _circle_ in circle_geoms:
        _svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#b0b0b0" />')

        for _other_ in circle_geoms:
            if _circle_ != _other_:
                _seg_   = (_circle_[:2], _other_[:2])
                _uv_    = rt.unitVector(_seg_)
                _perp_  = (-_uv_[1], _uv_[0])
                r_min   = min(_circle_[2],_other_[2])
                _seg_0_ = ((_circle_[0] + _perp_[0] * r_min, _circle_[1] + _perp_[1] * r_min), 
                           (_other_[0]  + _perp_[0] * r_min, _other_[1]  + _perp_[1] * r_min))
                _seg_1_ = ((_circle_[0] - _perp_[0] * r_min, _circle_[1] - _perp_[1] * r_min), 
                           (_other_[0]  - _perp_[0] * r_min, _other_[1]  - _perp_[1] * r_min))

                #_svg_.append(f'<line {lsvg(_seg_)}   stroke="black" stroke-width="0.2" />')
                #_svg_.append(f'<line {lsvg(_seg_0_)} stroke="red"   stroke-width="0.1" />')
                #_svg_.append(f'<line {lsvg(_seg_1_)} stroke="red"   stroke-width="0.1" />')

                no_intersects = True
                for _test_ in circle_geoms:
                    if _test_ != _circle_ and _test_ != _other_:
                        _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_,   _test_)
                        if _d_inter_ <= _test_[2]:
                            no_intersects = False
                            break
                        _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_0_, _test_)
                        if _d_inter_ <= _test_[2]:
                            no_intersects = False
                            break
                        _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_1_, _test_)
                        if _d_inter_ <= _test_[2]:
                            no_intersects = False
                            break
                        
                if no_intersects:
                    _pt_   = (_seg_[0][0] + _seg_[1][0])/2.0, (_seg_[0][1] + _seg_[1][1])/2.0
                    _svg_.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="1" fill="black" />')
                    _bone_ = ((_pt_[0] + _perp_[0] *r_min/2, _pt_[1] + _perp_[1] *r_min/2),
                              (_pt_[0] - _perp_[0] *r_min/2, _pt_[1] - _perp_[1] *r_min/2))
                    _svg_.append(f'<line {lsvg(_bone_)} stroke="black" stroke-width="0.2" />')

    _svg_.append('</svg>')
    return ''.join(_svg_)
rt.svgObject(circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_))

In [ ]:
#
# circularPathRouter() - route exits to a single entry around circles
# - all points need to be at least circle_radius + radius_inc_test + 1.0 from the circle centers...
#
def circularPathRouter(entry_pt,                  # (x,y,circle_i) -- where circle_i is the circle index from circle_geoms
                       exit_pts,                  # [(x,y,circle_i),(x,y,circle_i),(x,y,circle_i), ...] -- where circle_i is the circle index from circle_geoms
                       circle_geoms,              # [(cx,cy,r),(cx,cy,r), ...]
                       draw_skeleton      = False,# draw the skeleton of the exit paths
                       escape_px          = 5,    # length to push the exit points (and entry point) away from circle
                       min_circle_sep     = 30,   # minimum distance between circles
                       half_sep           = 15,   # needs to be more than the radius_inc_test ... half separation (but doesn't have to be)
                       radius_inc_test    = 4,    # for routing around circles, how much to test with
                       radius_start       = 5,    # needs to be more than the radius_inc_test ... less than the min_circle_sep
                       max_pts_per_node   = 50,   # maximum points per node for the xy quad tree
                       merge_distance_min = 5):   # minimum distance necessary to merge a path into an already exiting path
    lsvg = lambda seg: f' x1="{seg[0][0]}" y1="{seg[0][1]}" x2="{seg[1][0]}" y2="{seg[1][1]}" '
    _svg_ = ['<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />']
    skeleton_pts = []
    for _circle_ in circle_geoms:
        _svg_.append(f'<circle cx="{_circle_[0]}" cy="{_circle_[1]}" r="{_circle_[2]}" fill="none" stroke="#b0b0b0" />')
        for a in range(0,360,30):
            _x_ = _circle_[0] + (_circle_[2] + 2*radius_inc_test) * cos(a)
            _y_ = _circle_[1] + (_circle_[2] + 2*radius_inc_test) * sin(a)
            if draw_skeleton:
                _svg_.append(f'<circle cx="{_x_}" cy="{_y_}" r="2" fill="black" />')
            skeleton_pts.append((_x_,_y_))

        for _other_ in circle_geoms:
            if _circle_ != _other_:
                _seg_   = (_circle_[:2], _other_[:2])
                _uv_    = rt.unitVector(_seg_)
                _perp_  = (-_uv_[1], _uv_[0])
                r_min   = min(_circle_[2],_other_[2])
                _seg_0_ = ((_circle_[0] + _perp_[0] * r_min, _circle_[1] + _perp_[1] * r_min), 
                           (_other_[0]  + _perp_[0] * r_min, _other_[1]  + _perp_[1] * r_min))
                _seg_1_ = ((_circle_[0] - _perp_[0] * r_min, _circle_[1] - _perp_[1] * r_min), 
                           (_other_[0]  - _perp_[0] * r_min, _other_[1]  - _perp_[1] * r_min))

                no_intersects = True
                for _test_ in circle_geoms:
                    if _test_ != _circle_ and _test_ != _other_:
                        _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_,   _test_)
                        if _d_inter_ <= _test_[2]:
                            no_intersects = False
                            break
                        _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_0_, _test_)
                        if _d_inter_ <= _test_[2]:
                            no_intersects = False
                            break
                        _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_1_, _test_)
                        if _d_inter_ <= _test_[2]:
                            no_intersects = False
                            break
                        
                if no_intersects:
                    _pt_   = (_seg_[0][0] + _seg_[1][0])/2.0, (_seg_[0][1] + _seg_[1][1])/2.0
                    skeleton_pts.append(_pt_)
                    if draw_skeleton:
                        _svg_.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="3" fill="red" />')

                    _ext_ = _pt_[0] + _perp_[0] * r_min/2, _pt_[1] + _perp_[1] * r_min/2
                    skeleton_pts.append(_ext_)
                    if draw_skeleton:
                        _svg_.append(f'<circle cx="{_ext_[0]}" cy="{_ext_[1]}" r="1" fill="red" />')

                    _ext_ = _pt_[0] - _perp_[0] * r_min/2, _pt_[1] - _perp_[1] * r_min/2
                    skeleton_pts.append(_ext_)
                    if draw_skeleton:
                        _svg_.append(f'<circle cx="{_ext_[0]}" cy="{_ext_[1]}" r="1" fill="red" />')

    already_tested = set()
    _sorter_       = []
    for _pt_ in skeleton_pts:
        for _other_ in skeleton_pts:
            if _pt_ != _other_:
                _intersects_ = False
                _seg_ = (_pt_, _other_)
                _alt_ = (_other_, _pt_)
                if _seg_ in already_tested or _alt_ in already_tested:
                    continue
                already_tested.add(_seg_)
                already_tested.add(_alt_)
                for _circle_ in circle_geoms:
                    _d_inter_, _pt_inter_ = rt.segmentIntersectsCircle(_seg_, _circle_)
                    if _d_inter_ <= _circle_[2]+radius_inc_test:
                        _intersects_ = True
                        break
                if _intersects_ == False:
                    # _svg_.append(f'<line {lsvg(_seg_)} stroke="black" stroke-width="0.05" />')
                    _sorter_.append((rt.segmentLength(_seg_),_seg_))
    _sorter_.sort()
    _keepers_ = []
    for x in _sorter_:
        _seg_ = x[1]
        _intersects_ = False
        for _test_ in _keepers_:
            _flag_, _xinter_, _yinter_, _ts0_, _ts1_ = rt.segmentsIntersect(_seg_, _test_)
            if _flag_ and _ts0_ != 0.0 and _ts0_ != 1.0:
                _intersects_ = True
                break
        if _intersects_ == False:
            if draw_skeleton:
                _svg_.append(f'<line {lsvg(_seg_)} stroke="black" stroke-width="0.5" />')
            _keepers_.append(_seg_)

    print(f'{len(_keepers_)=}')

    x0, y0, x1, y1 = None, None, None, None
    g, pts_set = nx.Graph(), set()
    for _seg_ in _keepers_:
        g.add_edge(_seg_[0], _seg_[1], weight=rt.segmentLength(_seg_))
        pts_set.add(_seg_[0])
        pts_set.add(_seg_[1])
        x0 = _seg_[0][0] if x0 is None else min(x0, _seg_[0][0])
        y0 = _seg_[0][1] if y0 is None else min(y0, _seg_[0][1])
        x1 = _seg_[1][0] if x1 is None else max(x1, _seg_[1][0])
        y1 = _seg_[1][1] if y1 is None else max(y1, _seg_[1][1])
 
    qt = rt.xyQuadTree((x0,y0,x1,y1))
    qt.add(pts_set)

    p0 = qt.closest(entry_pt, 1)[0][1]
    _svg_.append(f'<circle cx="{entry_pt[0]}" cy="{entry_pt[1]}" r="3" fill="red" />')
    for _pt_ in exit_pts:
        _svg_.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="3" fill="green" />')
        p1 = qt.closest(_pt_, 1)[0][1]
        _path_ = nx.shortest_path(g, p0, p1)
        _path_.insert(0, p0)
        _path_.append(_pt_)
        _svg_.append(f'<path d="{rt.svgPathCubicBSpline(_path_, 1.0)}" stroke="{rt.co_mgr.getColor(str(_pt_))}" stroke-width="1" fill="none" />')

    _svg_.append('</svg>')
    return ''.join(_svg_)
rt.svgObject(circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_))